In [9]:
pip install okx

Note: you may need to restart the kernel to use updated packages.


# 要購買的價格

In [171]:
from okx.app import OkxSWAP
from okx.app import MarketSWAP
from okx.app import Market as Market_okx
from okx.api import Account
from okx.api import Trade
from okx_api import Market as Market_api
from pprint import pprint
import json
import os
import time
import math
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
KEY = os.getenv('key')
SECRET = os.getenv('secret')
PASSPHRASE = os.getenv('passphrase')

okxSWAP = OkxSWAP(
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
)

[SUCCESS] 设置持仓方式为双向持仓成功，posMode="long_short_mode"


In [67]:
market = Market_api(key = '', secret = '', passphrase = '', flag = '0')
# 获取所有产品行情信息
result = market.get_history_index_candles(instId = "INJ-USDT", bar = "1m", limit = "10")
pprint(result['data'])
    
val = (float(result['data'][0][1]) - float(result['data'][9][1]))/float(result['data'][9][1])*100 #最新資料減第十個資料漲跌百分比
# 如果是跌幅
# float(result['data'][9][1])*(1 - abs(val)/100)
# 如果是漲幅
# float(result['data'][2][1])*(1 + abs(val)/100)

[['1705231320000', '38.062', '38.062', '37.976', '37.997', '1'],
 ['1705231260000', '38.144', '38.144', '38.064', '38.064', '1'],
 ['1705231200000', '38.079', '38.142', '38.078', '38.142', '1'],
 ['1705231140000', '38.049', '38.081', '38.033', '38.078', '1'],
 ['1705231080000', '37.938', '38.052', '37.938', '38.052', '1'],
 ['1705231020000', '37.854', '37.932', '37.854', '37.931', '1'],
 ['1705230960000', '37.762', '37.852', '37.762', '37.852', '1'],
 ['1705230900000', '37.807', '37.807', '37.733', '37.763', '1'],
 ['1705230840000', '37.86', '37.86', '37.824', '37.824', '1'],
 ['1705230780000', '37.913', '37.918', '37.853', '37.857', '1']]


In [68]:
account = Account(
    # 交易账户需要秘钥
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
    # 实盘：0，虚拟盘：1
    flag='0',
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None, 
)
# 实例化trade
trade = Trade(
    # 交易模块需要秘钥
    key = KEY,
    secret = SECRET,
    passphrase = PASSPHRASE,
    # 实盘：0，虚拟盘：1
    flag='0',
    # 使用http和https代理，proxies={'http':'xxxxx','https:':'xxxxx'}，通requests中的proxies参数规则相同
    proxies={},
    # 转发：需搭建转发服务器，可参考：https://github.com/pyted/okx_resender
    proxy_host=None, 
)

In [87]:
# get_ticker = okxSWAP.market.get_ticker('INJ-USDT-SWAP')
# pprint(get_ticker)
# get_books = okxSWAP.market.get_books('INJ-USDT-SWAP', sz=10)
# pprint(get_books)

In [81]:
# config_result = account.get_config()
# config_result

In [ ]:
# 账户模式acctLv
# 1: 简单交易模式
# 2: 单币种保证金模式
# 3: 跨币种保证金模式
# 4: 组合保证金模式
# set_account_level_result = account.set_account_level(acctLv='2')
# set_account_level_result

In [28]:
# 持仓方式: long_short_mode：开平仓模式 net_mode：买卖模式(适用交割/永续)
set_position_mode_result = account.set_position_mode(posMode='long_short_mode')
set_position_mode_result

{'code': '0', 'data': [{'posMode': 'long_short_mode'}], 'msg': ''}

In [30]:
# 永续合约持仓
positions_result = account.get_positions(instType='SWAP')
positions_result

{'code': '0', 'data': [], 'msg': ''}

## 獲取帳戶資金

In [25]:
balance_result = account.get_balance(ccy='USDT')
balance_result['data'][0]['details'][0]['availBal']

'28.6306943513004'

In [70]:
# 永续合约 BTC-USDT-SWAP 全仓
leverage_info_result = account.get_leverage_info(instId='INJ-USDT-SWAP', mgnMode='cross')
leverage_info_result

{'code': '0',
 'data': [{'instId': 'INJ-USDT-SWAP',
   'lever': '1',
   'mgnMode': 'cross',
   'posSide': 'long'},
  {'instId': 'INJ-USDT-SWAP',
   'lever': '1',
   'mgnMode': 'cross',
   'posSide': 'short'}],
 'msg': ''}

## 設定槓桿

In [69]:
set_leverage_result = account.set_leverage(
    instId='INJ-USDT-SWAP', 
    mgnMode='isolated', 
    lever='2.5', 
    posSide='long'
)
set_leverage_result

{'code': '0',
 'data': [{'instId': 'INJ-USDT-SWAP',
   'lever': '2.5',
   'mgnMode': 'isolated',
   'posSide': 'long'}],
 'msg': ''}

## 開倉

In [129]:
# 以2U的价格，逐仓限价开多1张
set_order_swap_limit_open_result = trade.set_order(
    instId = 'INJ-USDT-SWAP',
    ordType = 'limit',
    tdMode = 'isolated',
    posSide = 'long',
    side = 'buy',
    # 用sz的價格買px個幣
    sz = '3', # 已成交 | 委託總量
    px = '1', # 成交均價 | 委託價
)
set_order_swap_limit_open_result

{'code': '0',
 'data': [{'clOrdId': '',
   'ordId': '666753341787488266',
   'sCode': '0',
   'sMsg': 'Order placed',
   'tag': ''}],
 'inTime': '1705238384360012',
 'msg': '',
 'outTime': '1705238384361984'}

In [169]:
set_order_swap_limit_open_result['data'][0]['ordId']

'666650289051025415'

In [56]:
order_result = trade.get_order(
    instId = 'BTC-USDT-SWAP',
    ordId = set_order_swap_limit_open_result['data'][0]['ordId']
)
order_result['data']#[0]['state']

[{'accFillSz': '0',
  'algoClOrdId': '',
  'algoId': '',
  'attachAlgoClOrdId': '',
  'attachAlgoOrds': [],
  'avgPx': '',
  'cTime': '1705227154877',
  'cancelSource': '',
  'cancelSourceReason': '',
  'category': 'normal',
  'ccy': '',
  'clOrdId': '',
  'fee': '0',
  'feeCcy': 'USDT',
  'fillPx': '',
  'fillSz': '0',
  'fillTime': '',
  'instId': 'BTC-USDT-SWAP',
  'instType': 'SWAP',
  'lever': '3',
  'ordId': '666706241922023431',
  'ordType': 'limit',
  'pnl': '0',
  'posSide': 'long',
  'px': '1',
  'pxType': '',
  'pxUsd': '',
  'pxVol': '',
  'quickMgnType': '',
  'rebate': '0',
  'rebateCcy': 'USDT',
  'reduceOnly': 'false',
  'side': 'buy',
  'slOrdPx': '',
  'slTriggerPx': '',
  'slTriggerPxType': '',
  'source': '',
  'state': 'live',
  'stpId': '',
  'stpMode': '',
  'sz': '3',
  'tag': '',
  'tdMode': 'isolated',
  'tgtCcy': '',
  'tpOrdPx': '',
  'tpTriggerPx': '',
  'tpTriggerPxType': '',
  'tradeId': '',
  'uTime': '1705227154877'}]

## 平倉

In [153]:
# 以2U的价格，逐仓限价平多1张个MANA-USDT-SWAP
set_order_swap_limit_close_result = trade.set_order(
    instId='MANA-USDT-SWAP',
    ordType='limit',
    tdMode='isolated',
    posSide='long',
    side='sell',
    sz='1',
    px='1',
)
set_order_swap_limit_close_result

{'code': '1',
 'data': [{'clOrdId': '',
   'ordId': '',
   'sCode': '51169',
   'sMsg': 'Order failed. You don’t have any positions in this contract that can be closed. ',
   'tag': ''}],
 'inTime': '1705213615052527',
 'msg': 'All operations failed',
 'outTime': '1705213615053733'}

## 取消訂單

In [163]:
# 获取币币交易未成单的订单列表
orders_pending_result = trade.get_orders_pending()
print(orders_pending_result['data'][0]['ordId'])

# # 按照instId与ordId取消订单
# set_cancel_order_result = trade.set_cancel_order(
#     instId = 'INJ-USDT-SWAP', ordId = orders_pending_result['data'][0]['ordId']
# )
# set_cancel_order_result

666649516720275456


## 市價全平

In [144]:
# MANA-USDT-SWAP 逐仓 多仓 市价全屏
set_close_position_result = trade.set_close_position(
    instId = 'INJ-USDT-SWAP',
    posSide = 'long',
    mgnMode = 'isolated'
)
set_close_position_result

{'code': '51023', 'data': [], 'msg': "Position doesn't exist"}

## 邏輯
1. 獲取當下價格計算和前幾分鐘前的價格差
2. 若價格漲跌超過0.3%，被視為不合理
3. 使用25%資金開倉
4. 追蹤開倉狀態，若沒有成功則取消訂單，反之設定止盈0.1%止損0.1%

In [102]:
result = market.get_history_index_candles(instId = "INJ-USDT", bar = "1m", limit = "20")
result['data']
#最新幣價對十根k線之前的幣價進行計算
val = (float(result['data'][0][1]) - float(result['data'][19][1]))/float(result['data'][19][1])*100
price_now = result['data'][0][1]

result['data'][1]# ts, 開盤, 最高, 最低, 收盤, k線狀態(0未完結, 1完結)
val

-0.12645555614100276

In [247]:
while True:
    # 標記一次開倉
    long_mark = True
    short_mark = True
    
    result = market.get_history_index_candles(instId = "INJ-USDT", bar = "1m", limit = "10")
    result['data']
    #最新幣價對十根k線之前的幣價進行計算
    val = (float(result['data'][0][1]) - float(result['data'][9][1]))/float(result['data'][9][1])*100
    price_now = result['data'][0][1]
    print(f"當下價格: {price_now}")
    print(f"震幅: {round(val, 3)}%")

    balance_result = account.get_balance(ccy = 'USDT')
    bal = balance_result['data'][0]['details'][0]['availBal']
    print(f"USDT balance: {bal}")

    # 設定每次交易只拿資產的 25% 操作
    # 總共可買入的幣，乘以 25%
    # 總共買入 px = total 個，價格為 result['data'][0][1]
    total = float(bal)/float(result['data'][0][1])*0.25
        
    # 確保資金永遠高於30u
    if float(bal) > 30:
        
        if val > 1.5:
            LEVER = 15
            
        elif val > 1:
            LEVER = 10
            
        elif val > 0.5:
            LEVER = 5
            
        else:
            LEVER = 1
            
        # 槓桿
        set_leverage_result = account.set_leverage(
            instId = 'INJ-USDT-SWAP', 
            mgnMode = 'isolated', 
            lever = LEVER, 
            posSide = 'long'
        )
        set_leverage_result = account.set_leverage(
            instId = 'INJ-USDT-SWAP', 
            mgnMode = 'isolated', 
            lever = LEVER, 
            posSide = 'short'
        )
        # 先確認能否開倉，判斷為有無任何訂單
        order_result = trade.get_order(
            instId = 'INJ-USDT-SWAP',
            ordId = set_order_swap_limit_open_result['data'][0]['ordId']
        )
        order_result['data'] = []
        # 沒有訂單則設定為可開倉標記
        if len(order_result['data']) == 0:

            short_mark = True
            long_mark = True

            print("可開倉")
        
        if val > 0.5 and short_mark:

            # 開空
            set_order_swap_limit_open_result = trade.set_order(
                instId = 'INJ-USDT-SWAP',
                ordType = 'limit',
                tdMode = 'isolated', # isolated:逐倉 cross:全倉
                posSide = 'short', # 空單對應到 sell
                side = 'sell',
                sz = str(math.floor(total*10)),
                px = str(float(result['data'][0][1])),
                tpTriggerPx = str(float(result['data'][0][1])*0.99), # 止盈
                tpOrdPx = '-1',
                slTriggerPx = str(float(result['data'][0][1])*1.01), # 止損
                slOrdPx = '-1'
            )

            # 等待10秒，看有沒有持倉
            time.sleep(10)
        
            # 追蹤訂單，重新呼叫get_order
            order_result = trade.get_order(
                instId = 'INJ-USDT-SWAP',
                ordId = set_order_swap_limit_open_result['data'][0]['ordId']
            )
            print('data', order_result['data'])
            
            # 若沒有成功持倉，撤單，等待下一個迴圈重新判斷
            if order_result['data'][0]['state'] == "live":
                
                # 按照instId与ordId取消订单
                set_cancel_order_result = trade.set_cancel_order(
                    instId = 'INJ-USDT-SWAP', ordId = set_order_swap_limit_open_result['data'][0]['ordId']
                )
                print("沒有成功持空倉，撤單")

            # 若成功持倉，設置平倉價格為當下的價格下跌0.1%，以及止損價格0.1%
            elif order_result['data'][0]['state'] == "filled":
                # 不會重複開倉
                long_mark = False
                
                print(f"已經設定空單止盈止損，槓桿為 {LEVER}")
                
            # 緩衝
            time.sleep(50)
            
            
        elif val < -0.5 and long_mark:
            
            # 開多
            set_order_swap_limit_open_result = trade.set_order(
                instId = 'INJ-USDT-SWAP',
                ordType = 'limit',
                tdMode = 'isolated', # isolated:逐倉 cross:全倉
                posSide = 'long', # 多單對應到 buy
                side = 'buy',
                sz = str(math.floor(total*10)),
                px = str(float(result['data'][0][1])),
                tpTriggerPx = str(float(result['data'][0][1])*1.01), # 止盈
                tpOrdPx = '-1',
                slTriggerPx = str(float(result['data'][0][1])*0.99), # 止損
                slOrdPx = '-1'
            )

            # 等待10秒，看有沒有持倉
            time.sleep(10)

            # 追蹤訂單，重新呼叫get_order
            order_result = trade.get_order(
                instId = 'INJ-USDT-SWAP',
                ordId = set_order_swap_limit_open_result['data'][0]['ordId']
            )
            print('data', order_result['data'])
            
            # 若沒有成功持倉，撤單，等待下一個迴圈重新判斷
            if order_result['data'][0]['state'] == "live":  # 需更改這裡的狀態結果，持倉並非filled
                
                # 按照instId与ordId取消订单
                set_cancel_order_result = trade.set_cancel_order(
                    instId = 'INJ-USDT-SWAP', ordId = set_order_swap_limit_open_result['data'][0]['ordId']
                )
                print("沒有成功持多倉，撤單")
                
            # 若成功持倉，設置平倉價格為當下的價格上漲1%，以及止損價格1%
            elif order_result['data'][0]['state'] == "filled":
                # 不會重複開倉
                short_mark = False

                print(f"已經設定空單止盈止損，槓桿為 {LEVER}")
        
            # 緩衝
            time.sleep(50)
            
        else:
            
            print("沒有動作")
            # 由於最低獲取間隔為1m，總緩衝需60秒
            time.sleep(60)
        

當下價格: 38.819
震幅: 0.429%
USDT balance: 93.0588664413004
沒有動作


KeyboardInterrupt: 

# 待完成: 如何在每個迴圈追蹤持有倉位並且設定標記